### Matrix Diagonalization
Consider a $n \times n$ matrix $A_{n, n}$ with $n$ linearly independent eigenvectors.
<br> Let $S_{n,n}$ be a matrix with these eigenvectors as its columns.
<br>Then $A_{n,n}$ can be factorized as
$$
A = S \Sigma S^{-1}
$$
where $\Sigma$ is a diagonal matrix (all off-digonal elements are zero).
<br> The above matrix factorization is known as diagonalization.

In [1]:
import torch

# Let us reconsider our rotation matrix
def diagonalise(matrix):
    try:
        # np.linalg.eig(matrix) returns the eigenvalues of matrix in an
        # array (first return value) and the eigvectors as a
        # matrix (each column is an eigenvector)
        l, e = torch.linalg.eig(matrix)
        
        # make a diagonal matrix from the eigenvalues
        sigma = torch.diag(l)
        
        # return the three factor matrices
        return e, torch.diag(l), torch.linalg.inv(e)
    except RuntimeError:
        print("Cannot diagonalise matrix!")

    
A = torch.tensor([[0.7071, 0.7071, 0], [-0.7071, 0.7071, 0],
              [0, 0, 1]])
print("A =\n{}".format(A))
S, sigma, S_inv = diagonalise(A)

# Let us reconstruct the original matriox from its factors
A1 = torch.matmul(S, torch.matmul(sigma, S_inv))
print("\nS =\n{}".format(S))
print("\nsigma =\n{}".format(sigma))
print("\nS_inv =\n{}".format(S_inv))
print("\nS sigmaa S_inv =\n{}".format(A1))

# We assert that the original matrix is the same as the
# reconstruction from the diagonal decomposition factors.

assert torch.allclose(A, A1.real)

A =
tensor([[ 0.7071,  0.7071,  0.0000],
        [-0.7071,  0.7071,  0.0000],
        [ 0.0000,  0.0000,  1.0000]])

S =
tensor([[0.7071+0.0000j, 0.7071-0.0000j, 0.0000+0.0000j],
        [0.0000+0.7071j, 0.0000-0.7071j, 0.0000+0.0000j],
        [0.0000+0.0000j, 0.0000-0.0000j, 1.0000+0.0000j]])

sigma =
tensor([[0.7071+0.7071j, 0.0000+0.0000j, 0.0000+0.0000j],
        [0.0000+0.0000j, 0.7071-0.7071j, 0.0000+0.0000j],
        [0.0000+0.0000j, 0.0000+0.0000j, 1.0000+0.0000j]])

S_inv =
tensor([[0.7071+0.0000j, 0.0000-0.7071j, 0.0000+0.0000j],
        [0.7071+0.0000j, 0.0000+0.7071j, 0.0000+0.0000j],
        [0.0000+0.0000j, 0.0000+0.0000j, 1.0000+0.0000j]])

S sigmaa S_inv =
tensor([[ 0.7071-5.5519e-08j,  0.7071+5.5519e-08j,  0.0000+0.0000e+00j],
        [-0.7071-5.5519e-08j,  0.7071-5.5519e-08j,  0.0000+0.0000e+00j],
        [ 0.0000+0.0000e+00j,  0.0000+0.0000e+00j,  1.0000+0.0000e+00j]])


#### Solving Linear Systems without Inverse

Let us try solving the following set of equations:

\begin{align*}
    x + 2y + z    &= 8 \\
    2x + 2y + 3z &= 15 \\
    x + 3y + 3z  &= 16
\end{align*}

This can be written using matrices and vectors as
\begin{equation*}
A\vec{x} = \vec{b}
\end{equation*}
where $A=
\begin{bmatrix}
        1 & 2 & 1 \\
        2 & 2 & 3 \\
        1 & 3 & 3 
\end{bmatrix}
\;\;\;\;\;\;
\vec{x} = 
\begin{bmatrix}
        x \\
        y \\
        z 
\end{bmatrix}
\;\;\;\;\;\;
\vec{b} = 
\begin{bmatrix}
        8 \\
        15 \\
        16
\end{bmatrix}$


Note that $A$ is a symmetric matrix. It has orthogonal eigenvectros.
<br> The matrix with eigenvectors of $A$ in columns is orthogonal.
<br> Its transpose and inverse are same.

In [2]:
A = torch.tensor([[1, 2, 1], [2, 2, 3], [1, 3, 3]], dtype=torch.float)
assert torch.all(A == A.T)
b = torch.tensor([8, 15, 16], dtype=torch.cfloat)

# One way to solve for this is to compute matrix inverse
# i.e x = A_inv b
x_0 = torch.matmul(torch.linalg.inv(A), b.real)
print("Solution using inverse: {}".format(x_0))

# Matrix inversion is a complex process that can be
# numerically unstable. If possible we use diagonalisation.
w, S = torch.linalg.eig(A)

# We know that A = S sigma S_inv
# So S sigma S_inv x = b

# sigma S_inv x =  S_inv b  ==> sigma S_inv x = S_t b
S_inv_b = torch.matmul(S.T, b)

# => S_inv x = sigma_inv(S_t b)
# Since inversion of the diagonal matrix is just division
# of all elements by 1, we can compute 
# sigma_inv as follows
sigma_inv = torch.diag(1/ w)
sigma_inv_S_inv_b = torch.matmul(sigma_inv, S_inv_b)

# => x = S (sigma_inv(S_t b))
x_1 = torch.matmul(S, sigma_inv_S_inv_b) 
print("Solution using diagonalisation: {}".format(x_1))

assert torch.allclose(x_0, x_1.real)

Solution using inverse: tensor([1.0000, 2.0000, 3.0000])
Solution using diagonalisation: tensor([1.0000+0.j, 2.0000+0.j, 3.0000+0.j])


#### Matrix powers using diagonalization

In [3]:
A = torch.tensor([[1, 2, 1], [2, 2, 3], [1, 3, 3]], dtype=torch.float)

def brute_force_matrix_power(matrix, y):
    """Returns matrix raised to the power of y"""
    # Assert that it is a square matrix
    assert len(matrix.shape) == 2\
           and matrix.shape[0] == matrix.shape[1] 
    assert type(y) == int 
    if y == 0:
        # Return identity matrix 
        return torch.eye(matrix.shape[0])
    output_matrix = torch.clone(matrix)
    for i in range(y-1):
        output_matrix = torch.matmul(matrix, output_matrix)
    return output_matrix

def matrix_power_using_diag(matrix, y):
    """Returns matrix raised to the power of y"""
    # Assert that it is a square matrix
    assert len(matrix.shape) == 2\
           and matrix.shape[0] == matrix.shape[1] 
    assert type(y) == int 
    if y == 0:
        # Return identity matrix 
        return torch.eye(matrix.shape[0])
    w, v = torch.linalg.eig(matrix)
    w_y = w ** float(y) # Compute w^y
    return torch.matmul(v, torch.matmul(torch.diag(w_y),
                                  torch.linalg.inv(v)))
    
# One way to compute A**2 would be to multiply A by A
# This is an O(nm^3) operation
A_2_brute_force = brute_force_matrix_power(A, 2)


# We can instead diagonalise the matrix and compute A ** 2
# This reduces the complexity to O(m)
A_2_diag = matrix_power_using_diag(A, 2)

print("A:\n{}".format(A))
print("A**2 computed via brute force:\n{}".\
      format(A_2_brute_force))
print("A**2 computed using diagonalisation:\n{}".\
      format(A_2_diag))

assert torch.allclose(A_2_brute_force, A_2_diag.real)

A:
tensor([[1., 2., 1.],
        [2., 2., 3.],
        [1., 3., 3.]])
A**2 computed via brute force:
tensor([[ 6.,  9., 10.],
        [ 9., 17., 17.],
        [10., 17., 19.]])
A**2 computed using diagonalisation:
tensor([[ 6.0000+0.j,  9.0000+0.j, 10.0000+0.j],
        [ 9.0000+0.j, 17.0000+0.j, 17.0000+0.j],
        [10.0000+0.j, 17.0000+0.j, 19.0000+0.j]])
